# マルコフ連鎖
人工無能の一番簡単な例.  
大規模なツイート情報をくれた友人に感謝

In [13]:
# -*- coding: utf-8 -*-
import pandas as pd
import MeCab

from collections import Counter
import json
import os,re,random

import numpy as np

In [2]:
!head TWEETandURL.csv

text
寿司のフレンズと寿司会
今回は嫁も一緒に( ◜ω‾)
すしまみれ 新宿セントラルロード店 
あまりにも勉強しなさ過ぎて開きっぱなしの問題集にホコリが乗ってるの草
「休みだから家になっつんがいる嬉しい〜！」って騒いでたらカーペットにお弁当丸ごとひっくり返した......
 あるこさんおはようございます( ◜ω‾)
だってやってらんないじゃん
ストレスよりロマンスでしょ
 おはようございます( ◜ω‾)


In [3]:
filename="TWEETandURL.csv" #任意のファイル名
nrt_tweet = pd.read_csv("TWEETandURL.csv",encoding='utf-8')["text"][1:]
words = []
for _ in nrt_tweet:
    words.append(_.strip()+"<EOS>")
print("test: ",words[0])

test:  今回は嫁も一緒に( ◜ω‾)<EOS>


In [4]:
# -*- coding: utf-8 -*-
tmp=[]
#mt = MeCab.Tagger("-Ochasen")
mt = MeCab.Tagger(' -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
mt.parse("") #おまじない
for i,tweet in enumerate(nrt_tweet[:100]):
    #print(tweet,type(tweet))
    node = mt.parseToNode(tweet.strip())
    while node:
        ch = node.surface
        if ch == "":
            node = node.next
            continue
        tmp.append(ch)
        node = node.next
print(len(tmp))

1228


In [5]:
#マルコフ連鎖の辞書
def make_dic(words,dic):
    tmp = ["START"]
    for word in words:
        #該当文字が来たら次の文字ヘスキップ
        if word=="" or word=="\n\t" or word=="\n":
            continue
        #3wordsになるまで文字を加える
        tmp.append(word)
        if len(tmp) < 3: 
            continue
        #4単語 tmpに格納されているので一番古い一単語を除去
        if len(tmp) > 3:
            tmp = tmp[1:]
        set_word3(dic,tmp)
        if word == "EOS":
            tmp = ["START"]
            return dic
            #continue
    #print(dic)
    return dic

In [7]:
def set_word3(dic,s3):
    w1,w2,w3 = s3
    if not w1 in dic:
        dic[w1] = {}
    if not w2 in dic[w1]:
        dic[w1][w2] = {}
    if not w3 in dic[w1][w2]:
        dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1

In [8]:
#挙動テスト
test_dic={}
print("D===============\n{}\n".format(test_dic))
test_dic = make_dic(["お腹","が","今","とても","空いた","助けてくれ","EOS"],test_dic)
print("D================\n{}\n".format(test_dic))
test_dic = make_dic(["今","ハンバーガー","が","今","食べたい","EOS"],test_dic)
print("D================\n{}\n".format(test_dic))

D===============
{}

D================
{'START': {'お腹': {'が': 1}}, 'お腹': {'が': {'今': 1}}, 'が': {'今': {'とても': 1}}, '今': {'とても': {'空いた': 1}}, 'とても': {'空いた': {'助けてくれ': 1}}, '空いた': {'助けてくれ': {'EOS': 1}}}

D================
{'START': {'お腹': {'が': 1}, '今': {'ハンバーガー': 1}}, 'お腹': {'が': {'今': 1}}, 'が': {'今': {'とても': 1, '食べたい': 1}}, '今': {'とても': {'空いた': 1}, 'ハンバーガー': {'が': 1}, '食べたい': {'EOS': 1}}, 'とても': {'空いた': {'助けてくれ': 1}}, '空いた': {'助けてくれ': {'EOS': 1}}, 'ハンバーガー': {'が': {'今': 1}}}



In [9]:
#ツイートの自動生成
def make_tweet(dic,top_word="START"):
    ret = []
    if not top_word in dic:
        return "no dic"
    #top = dic["START"]
    top = dic[top_word]
    w1 = word_choice(top)
    if top_word != "START":
        w1 = top_word
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    flag = True
    if w1 == "EOS" or w2 == "EOS":
        flag = False
    while flag:
        #print(dic[w1][w2])
        try:
            #print("CHECK1: ",w1,w2)
            #print("CHECK2: ",dic[w1][w2])
            w3 = word_choice(dic[w1][w2])
            if w3 == "EOS":
                break
            ret.append(w3)
            w1,w2 = w2,w3
        except {KeyError,TypeError} as e:
            print(e)
            #print(dic[w1][w2])
    return "".join(ret)

In [10]:
def word_choice(sel):
    keys = sel.keys()
    L = random.choice(list(keys))
    #print(keys,L)
    return L

In [11]:
def main(tweets,markov_file=None):
    '''
    tweets : 全ツイート
    markov_file : 辞書ですね
    '''
    #dict_file = markov_file
    #if os.path.exists(dict_file):
    #    return json.load(open(dict_file,"r"))
    tagger = MeCab.Tagger('-Owakati')
    #tagger = MeCab.Tagger(' -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    dic={}
    for i,tweet in enumerate(tweets):
        #print(i)
        nodes = tagger.parseToNode(tweet)
        #辞書を作成する
        #dic = make_dic(nodes)
        words=[""] #分かち書きした単語をリストに加える
        while nodes:
            if nodes.surface=="":
                nodes = nodes.next
            else:
                words.append(nodes.surface)
                nodes = nodes.next
        words.append("EOS")
        dic = make_dic(words,dic) #マルコフ連鎖の辞書へ
        #json.dump(dic,open(dict_file,"w"))
    return dic
        

In [14]:
#辞書格納 and ファイルに書き込むとき用
dic = main(nrt_tweet)
json.dump(dic,open("markov_dict.json","w"))
print("書き込み終了")

書き込み終了


In [16]:
#f=open("NRT_Markov_Result.csv","w")
for i in range(10):
    print(i+1,make_tweet(dic).replace("EOS",""),sep=",")
#f.close()

1,灰になっちゃったりしない系ツイートかと詰め込んだようだ！😄✴✨」
2,買おうとおもいまふぁみふぁみまー♪しましたは現実世界は言葉が出ない妥当性を示唆
3,デレて
4,ケトル以外高過ぎて俺とLINEする人，バンバン留年した失敗しました😌🙌🏻
5,中高体操部ってオタクおすすめの牛角新宿歌舞伎町で黒人のゴツイにーがー＼＼\└('ω')┘//／／!!
6,....oh........nothankyou！なんか髪の毛もじゃもじゃでｵﾃﾝｺみたいなモブいた据え置きのパソコンが勝手に載るのな(･へ･)
7,話し相手が居たのでホイホイ付いていどねぇ......(ﾁﾗﾁﾗﾁﾗﾁﾗﾁﾗ)
8,ぬるぬる動くあきやまさん
9,ダックイン
10,幅広いジャンルで活用できるって、切りどころそこじゃない本は研究室面接対策
